In [1]:
import underworld3 as uw
import numpy as np
import sympy
import mpi4py

In [2]:
##mesh = uw.meshing.Annulus(radiusOuter = 1.0, radiusInner = 0.3, cellSize=0.1, qdegree=3)
mesh = uw.meshing.UnstructuredSimplexBox(minCoords=(0,0), maxCoords=(1,1), cellSize=0.1, qdegree=3)

In [3]:
v = uw.discretisation.MeshVariable("U", mesh, mesh.dim, degree=2)
p = uw.discretisation.MeshVariable("P", mesh, 1, degree=1)

In [4]:
stokes = uw.systems.Stokes(mesh, velocityField=v, pressureField=p)
stokes.constitutive_model = uw.systems.constitutive_models.ViscousFlowModel(mesh.dim)
stokes.constitutive_model.Parameters.viscosity = 1

In [5]:
stokes.penalty = 0.0
stokes.bodyforce = sympy.Matrix([0, 0])
stokes.saddle_preconditioner = 1 / stokes.constitutive_model.Parameters.viscosity

In [6]:
stable =  (0, 0)
dynamic = (1, 0)

stokes.add_dirichlet_bc(
    stable, ["Left", "Right", "Bottom"], (0, 1)
)
stokes.add_dirichlet_bc(
    dynamic, "Top", (0, 1)
)

In [7]:
stokes.solve()

  0 SNES Function norm 7.3056 
  1 SNES Function norm 4.05967e-09 
Nonlinear Stokes_1_ solve converged due to CONVERGED_FNORM_RELATIVE iterations 1


3

In [8]:
if mpi4py.MPI.COMM_WORLD.size == 1:
    import numpy as np
    import pyvista as pv
    import vtk
    
    pv.start_xvfb()
    pv.global_theme.background = "white"
    pv.global_theme.window_size = [750, 1200]
    pv.global_theme.antialiasing = True
    pv.global_theme.jupyter_backend = "panel"
    pv.global_theme.smooth_shading = True
    mesh.vtk("tmp_mesh.vtk")
    pvmesh = pv.read("tmp_mesh.vtk")
    pvmesh.point_data["P"] = uw.function.evaluate(p.sym[0], mesh.data)
    pvmesh.point_data["V"] = uw.function.evaluate(v.sym.dot(v.sym), mesh.data)
    arrow_loc = np.zeros((stokes.u.coords.shape[0], 3))
    arrow_loc[:, 0:2] = stokes.u.coords[...]
    arrow_length = np.zeros((stokes.u.coords.shape[0], 3))
    arrow_length[:, 0] = uw.function.evaluate(stokes.u.sym[0], stokes.u.coords)*0.05
    arrow_length[:, 1] = uw.function.evaluate(stokes.u.sym[1], stokes.u.coords)*0.05
    pl = pv.Plotter(window_size=[1000, 1000])
    pl.add_axes()
    pl.add_mesh(
        pvmesh,
        cmap="coolwarm",
        edge_color="Black",
        show_edges=True,
        scalars="V",
        use_transparency=False,
        opacity=1.0,
    )
    pl.add_arrows(arrow_loc, arrow_length, mag=3)
    pl.show(cpos="xy")

/Users/maxwilliams/anaconda3/envs/uw3/lib/python3.10/site-packages/pyvista/themes.py:2577: PyVistaDeprecationWarning: antialising is deprecated.  Please use `anti_aliasing` instead.
  warnings.warn(
/Users/maxwilliams/anaconda3/envs/uw3/lib/python3.10/site-packages/pyvista/themes.py:2545: PyVistaDeprecationWarning: `anti_aliasing` is now a string or None and must be either "ssaa", "msaa", "fxaa", or None
  warnings.warn(


/var/folders/s9/rd03rq4x0px79z1jnyqlz3540000gn/T/ipykernel_3281/744855170.py:10: PyVistaDeprecationWarning: `panel` backend is deprecated and is planned for future removal.
  pv.global_theme.jupyter_backend = "panel"


BokehModel(combine_events=True, render_bundle={'docs_json': {'c82fb155-c938-4b2e-84a4-8ee40e6f50e2': {'version…

: 